## Differential equations
$\dfrac{dS}{dt} = -\tau(t) S(t)[I(t) + U(t)] $ <br />
<br />
$\dfrac{dI}{dt} = \tau(t) S(t)[I(t) + U(t)] - \nu I(t)$ <br />
<br />
$\dfrac{dR}{dt} = \nu_{1} I(t) - \eta R(t)$ <br />
<br />
$\dfrac{dU}{dt} = \nu_{2} I(t) - \eta U(t)$ <br />

# Importing Libraries

In [13]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go 
import datetime
from ipywidgets import interact
import ipywidgets as wg
from ipywidgets import FloatSlider
from IPython.display import display
%matplotlib inline 

In [14]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

<IPython.core.display.Javascript object>

# Common Functions

In [15]:
# calculate Initial population in the I and U compartments
def Cal_Init_I0U0(S0,x2,f,v1,v2,eta):
    """
    Calculates the initial number of asymptomatic infectious (I0) and unreported symptomatic infectious (U0) at time t0.

    Parameters:
        S0 (float): Initial population in the susceptible compartment.
        x2 (float): Recovery rate.
        f (float): Fraction of asymptomatic infections.
        v1 (float): Reporting rate of symptomatic infections.
        v2 (float): Reporting rate of unreported symptomatic infections.
        eta (float): Clearance rate of unreported symptomatic infections.

    Returns:
        I0 (float): Initial number of asymptomatic infectious at time t0.
        U0 (float): Initial number of unreported symptomatic infectious at time t0.
        S0 (float): Initial population in the susceptible compartment.
    """
    I0 = x2/(f*(v1+v2))                       #number of asymptomatic infectious at time t0
    U0 = ((1-f)*(v1 + v2) / (eta + x2)) * I0  #number of unreported symptomatic infectious at time t0
    return I0, U0, S0

# transmission rate at time t0
def Ti_0(x2, v1, v2, S, eta):
    """
    Calculates the transmission rate at time t0.

    Parameters:
        x2 (float): Recovery rate.
        v1 (float): Reporting rate of symptomatic infections.
        v2 (float): Reporting rate of unreported symptomatic infections.
        S (float): Population in the susceptible compartment.
        eta (float): Clearance rate of unreported symptomatic infections.

    Returns:
        float: Transmission rate at time t0.
    """
    return ((x2 + v1 + v2) / S)*((eta + x2) / (v2 + eta + x2))


# time dependent transmission rate
def Ti(t, N, x2, v1, v2, S, eta, Mu): 
    """
    Calculates the time-dependent transmission rate.

    Parameters:
        t (float): Time.
        N (float): Time of intervention.
        x2 (float): Recovery rate.
        v1 (float): Reporting rate of symptomatic infections.
        v2 (float): Reporting rate of unreported symptomatic infections.
        S (float): Population in the susceptible compartment.
        eta (float): Clearance rate of unreported symptomatic infections.
        Mu (float): Decay rate of intervention effect.

    Returns:
        float: Time-dependent transmission rate.
    """
    if t <= N:
        retval = ((x2 + v1 + v2) / S)*((eta + x2) / (v2 + eta + x2))
    else:
        retval = (((x2 + v1 + v2) / S)*((eta + x2) / (v2 + eta + x2)))* math.exp(-Mu*(t - N))
    return retval


# transmission rate over time
def Ti_over_time(R0_ti_t, N, x2, v1, v2, S, eta, Mu):
    """
    Calculates the transmission rate over time.

    Parameters:
        R0_ti_t (list): List of time-dependent basic reproduction number.
        N (float): Time of intervention.
        x2 (float): Recovery rate.
        v1 (float): Reporting rate of symptomatic infections.
        v2 (float): Reporting rate of unreported symptomatic infections.
        S (float): Population in the susceptible compartment.
        eta (float): Clearance rate of unreported symptomatic infections.
        Mu (float): Decay rate of intervention effect.

    Returns:
        list: Transmission rate over time.
    """
    return [Ti(i, N, x2, v1, v2, S, eta, Mu) for i in range(len(R0_ti_t))]

# basic reproduction number at time t0
def R_0(ti0, v1, v2, S, eta):
    """
    Calculates the basic reproduction number at time t0.

    Parameters:
        ti0 (float): Transmission rate at time t0.
        v1 (float): Reporting rate of symptomatic infections.
        v2 (float): Reporting rate of unreported symptomatic infections.
        S (float): Population in the susceptible compartment.
        eta (float): Clearance rate of unreported symptomatic infections.

    Returns:
        float: basic reproduction number at time t0
    """
    return ((ti0 * S)/(v1 + v2)) * (1 + (v2 / eta))

# basic reproduction number over time
def R_0_over_time(t, N, x2, v1, v2, S, eta, Mu):
    tis = [Ti(i, N, x2, v1, v2, S, eta, Mu) for i in range(len(t))]
    r0s = [R_0(tis[i], v1, v2, S, eta) for i in range(len(tis))]
    return r0s

# Cumulative Data

In [16]:
def CumRep(t: float, t0: float, dIdt: float, N: float, x1: float, x2: float, x3: float) -> float:
    """
    Calculate the cumulative reported and unreported cases.

    Args:
        t (float): Time point for which to calculate the cumulative cases.
        t0 (float): Starting time point for the calculation.
        dIdt (float): The instantaneous rate of increase in cases.
        N (float): The end time point for the calculation.
        x1 (float): An exponential growth parameter.
        x2 (float): An exponential growth parameter.
        x3 (float): A constant offset for the exponential growth function.

    Returns:
        float: The calculated cumulative cases.
    """
    if t <= N:
        retval = x1*math.exp(x2*t)-x3
    else:
        retval = dIdt
    return retval

# differential equations
def deriv_Cum(y: tuple, t: float, Ti: callable, v: float, eta: float, v1: float, v2: float,
              f: callable, t0: float, N: float, x2: float, Mu: float, x1: float) -> tuple:
    """
    Calculate the derivative of the state variables for the SEIR model with cumulative reported and unreported cases.

    Args:
        y (tuple): A tuple containing the initial values of the state variables (S, I, U, CR, CU).
        t (float): The current time point for the calculation.
        Ti (callable): A function that calculates the transmission rate.
        v (float): The recovery rate for infected individuals.
        eta (float): The recovery rate for unreported infected individuals.
        v1 (float): A parameter for the transmission rate function.
        v2 (float): A parameter for the transmission rate function.
        f (callable): A function that calculates the time-varying death rate.
        t0 (float): The starting time point for the calculation.
        N (float): The end time point for the calculation.
        x2 (float): An exponential growth parameter for the transmission rate function.
        Mu (float): The natural mortality rate.
        x1 (float): An exponential growth parameter for the cumulative reported and unreported cases.

    Returns:
        tuple: A tuple containing the calculated derivatives for the state variables (dSdt, dIdt, dUdt, dCRdt, dCUdt).
    """
    S, I, U, CR, CU = y
    dSdt = -Ti(t, N, x2, v1, v2, S, eta, Mu) * S *(I + U)
    dIdt = Ti(t, N, x2, v1, v2, S, eta, Mu) * S * (I + U) - v * I
    dUdt = v2 * I - eta * U
    CR = I
    CR += CumRep(t, t0, dIdt,N,x1,x2,1)
    CU = I + U
    CU += CumRep(t, t0, dIdt,N,x1,x2,1)
    return dSdt, dIdt, dUdt, CR, CU


def Calc_Cum(t, S0, x2, v, eta, v1, v2, f, t0, N, Mu,x1):
    """
    Calculate the cumulative reported and unreported cases.
    
    Args:
        t: array of time points to calculate the solution for
        S0: initial susceptible population
        x2: contact rate of infectious individuals
        v: rate of recovery or death
        eta: rate of transition of asymptomatic to symptomatic
        v1: proportion of symptomatic cases that are reported
        v2: proportion of asymptomatic cases that are reported
        f: reduction factor for the contact rate due to interventions
        t0: time of intervention implementation
        N: total population size
        Mu: excess death rate
        x1: growth rate of cumulative cases
    
    Returns:
        The function returns the cumulative reported and unreported cases over time
    """
    # Initial population in the different compartments
    I0, U0, S0 = Cal_Init_I0U0(S0,x2,f,v1,v2,eta)
    CR0 = 1.0   #number of cumulated reported symptomatic infectious at time t0
    CU0 = 1.0   #number of cumulated unreported symptomatic infectious at time t0
    
    y0 = [S0, I0, U0, CR0, CU0]    #initial population in different compartments
    print('\033[1m Initial Values: \033[0m')
    print ('I\N{SUBSCRIPT ZERO} = ' + "{:.2f}".format(I0))
    print ('U\N{SUBSCRIPT ZERO} = ' + "{:.2f}".format(U0))
    print('N = ' + str(N))
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv_Cum, y0, t, args=(Ti, v, eta, v1, v2, f, t0, N, x2, Mu,x1))
    S, I, U, CR, CU= ret.T
    CR = v1 * CR
    CU = v2 * CU
    return CR, CU

# Daily Reported Cases

In [17]:
# differential equation
def deriv_Daily(y, t, Ti, v, eta, v1, v2 ,f, N, x2, Mu):
    """
    This function returns the derivatives of the Daily SIR model for a given time step.

    Args:
    - y (list): list of initial conditions for the differential equation.
    - t (float): time step.
    - Ti (function): function for calculating the transmission rate.
    - v (float): removal rate of infectious individuals.
    - eta (float): transition rate from the unreported symptomatic compartment to the reported symptomatic compartment.
    - v1 (float): transition rate from the asymptomatic compartment to the reported symptomatic compartment.
    - v2 (float): transition rate from the asymptomatic compartment to the unreported symptomatic compartment.
    - f (float): fraction of symptomatic infections that are reported.
    - N (float): number of days to consider.
    - x2 (float): initial number of asymptomatic infectious individuals.
    - Mu (float): daily recovery rate.

    Returns:
    - (tuple): a tuple of derivative values for the Daily SIR model.
    """
    S, I,  U ,DR = y
    dSdt = -Ti(t, N, x2, v1, v2, S, eta, Mu) * S *(I + U)
    dIdt = Ti(t, N, x2, v1, v2, S, eta, Mu) * S * (I + U) - v * I
    dUdt = v2 * I - eta * U
    dDRdt = (v * f * I) - DR
    return dSdt, dIdt, dUdt ,dDRdt

def Calc_Daily(t, S0, x2, v, eta, v1, v2, f, RealDataDR, N, Mu):
    """
    This function calculates the Daily SIR model for a given set of parameters.

    Args:
    - t (array): array of time values.
    - S0 (float): initial susceptible population.
    - x2 (float): initial number of asymptomatic infectious individuals.
    - v (float): removal rate of infectious individuals.
    - eta (float): transition rate from the unreported symptomatic compartment to the reported symptomatic compartment.
    - v1 (float): transition rate from the asymptomatic compartment to the reported symptomatic compartment.
    - v2 (float): transition rate from the asymptomatic compartment to the unreported symptomatic compartment.
    - f (float): fraction of symptomatic infections that are reported.
    - RealDataDR (array): array of real reported symptomatic infections.
    - N (float): number of days to consider.
    - Mu (float): daily recovery rate.

    Returns:
    - (tuple): a tuple of two arrays, the first being the calculated reported symptomatic infections and the second being the real reported symptomatic infections.
    """
    # Initial population in the I and U compartments
    I0, U0, S0 = Cal_Init_I0U0(S0,x2,f,v1,v2,eta)
    DR0 = 1.0   #daily number of reported symptomatic infectious at time t0

    y0 = [S0, I0, U0, DR0]            #initial population in different compartments
    
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv_Daily, y0, t, args=(Ti, v, eta, v1, v2,f, N, x2, Mu))
    S, I,  U, DR = ret.T
    
    return DR, RealDataDR

# Epidemic Curve

In [18]:
# differential equation
def deriv(y, t, Ti, v, eta, v1, v2, N, x2, Mu):
    """
    Calculate the derivatives of the SIRU model compartments.

    Parameters:
    y (list): list of initial values for the compartments [S, I, R, U]
    t (float): current time step
    Ti (function): function that returns the transmission rate at a given time
    v (float): removal rate of infectious individuals
    eta (float): removal rate of asymptomatic individuals
    v1 (float): rate of transition from infectious to removed (reported)
    v2 (float): rate of transition from infectious to asymptomatic
    N (float): total population size
    x2 (float): proportion of the population that is asymptomatic
    Mu (float): mutation rate
    
    Returns:
    dSdt (float): derivative of susceptible compartment
    dIdt (float): derivative of infectious compartment
    dRdt (float): derivative of removed compartment
    dUdt (float): derivative of asymptomatic compartment
    """
    S, I, R, U  = y
    dSdt = -Ti(t, N, x2, v1, v2, S, eta, Mu) * S *(I + U)
    dIdt = Ti(t, N, x2, v1, v2, S, eta, Mu) * S * (I + U) - v * I
    dRdt = v1 * I - eta * R
    dUdt = v2 * I - eta * U
    return dSdt, dIdt, dRdt, dUdt 

def Calc_All(t, S0, x2, v, eta, v1, v2, f, N, Mu):
    """
    Integrate the SIRU model differential equations over the time grid, t.

    Parameters:
    t (array-like): time grid
    S0 (float): initial value of the susceptible compartment
    x2 (float): proportion of the population that is asymptomatic
    v (float): removal rate of infectious individuals
    eta (float): removal rate of asymptomatic individuals
    v1 (float): rate of transition from infectious to removed (reported)
    v2 (float): rate of transition from infectious to asymptomatic
    f (float): proportion of infectious individuals that are reported
    N (float): total population size
    Mu (float): mutation rate
    
    Returns:
    S (array-like): time series of susceptible compartment
    I (array-like): time series of infectious compartment
    R (array-like): time series of removed compartment
    U (array-like): time series of asymptomatic compartment
    """
    # Initial population in the I and U compartments
    I0, U0, S0 = Cal_Init_I0U0(S0,x2,f,v1,v2,eta)
    R0 = 1.0   # number of reported symptomatic infectious at time t0
    
    y0 = [S0, I0, R0, U0]            #initial population in different compartments

    ret = odeint(deriv, y0, t, args=(Ti, v, eta, v1, v2, N, x2, Mu))
    S, I, R, U = ret.T
    return S, I, R, U

# Initialize Parameters

In [19]:
# initialize parameters values
def init_param(country,f):
    if country == "Germany":
        x1 = 50.71333
        x2= 0.197878
        x3 = 1
        Mu = 0.05
        S0 = 82.79e6 #number of susceptible at time t0
        N=  43
        v = 1/7
        eta = 1/7
        #data from Feb 1. until 13 April
        s_date = [datetime.datetime.strptime('1/02/20 01:55:19', '%d/%m/%y %H:%M:%S')]
        RealDataDR =[2,1,1,2,0,0,1,1,0,0,0,2,2,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4,26,10,54,18,28,39,66,138,284,163,55,237,157,271,733,1043,1174,
                         1144,1042,5940,4049,3276,3311,4438,2342,4954,5780,6294,3965,4751,4615,5453,6156,6174,6082,5936,
                         3677,3834,4003,4974,5323,4133]
        RealDataCum=[7,8,9,11,11,11,12,13,13,13,13,15,15,15,15,15,15,15,15,15,15,17,21,47,57,111,129,157,196,262,400,684,847,902,1139,1296,1567,3795,
                         4838,6012,7156,8198,14138,18187,21463,24774,29212,31554,36508,42288,48582,52547,57298,61913,67366,
                         73522,79696,85778,91714,95391,99225,103228,108202,113525,117658,120479,123016]
    if country == "Italy":
        x1 =156.321 
        x2= 0.20491 
        x3 = 1 
        Mu = 0.175
        S0 = 60.48e6 #number of susceptible at time t0
        N = 46
        v = 1/27
        eta = 1/14
        s_date = [datetime.datetime.strptime('28/01/20 01:55:19', '%d/%m/%y %H:%M:%S')]
        #data start from Jan. 28 until April 30.
        RealDataDR =[0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14,62,53,97,93,78,250,238,240,561,347,466,587,769,778,
                         1247,1492,1797,977,2313,2651,2547,3497,
                         2823,4000,3526,4207,5322,5986,6557,5560,4789,5249,5210,6153,5959,5974,5217,4050,4053,4782,4668,4585,
                         4805,4316,3599,3039,3836,4204,3951,4694,4092,3153,2972,2667,3786,3493,3491,3047,2256,2729,3370,2646,
                         3021,2357,2324,1739,2091,2086]
        RealDataCum=[3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,17,79,132,229,322,400,650,888,1128,1689,2036,2502,3089,3858,4636,5883,
                         7375,9172,10149,12462,15113,17660,21157,23980,27980,31506,35713,41035,47021,
                         53578,59138,63927,69176,74386,80539,86498,92472,97689,101739,105792,110574,
                         115242,119827,124632,128948,132547,135586,139422,143626,147577,152271,156363,
                         159516,162488,165155,168941,172434,175925,178972,181228,183957,187327,189973,
                         192994,195351,197675,199414,201505,203591]
    if country == "South Korea":
        x1 = 0.624
        x2= 0.294
        x3 = 1
        Mu = 2.4
        S0 = 51.47e6 #number of susceptible at time t0(South Korea population)
        N = 27
        v = 1/7
        eta = 1/7
        # data start from Feb.2, 2020 until March 29.
        s_date = [datetime.datetime.strptime('2/02/20 01:55:19', '%d/%m/%y %H:%M:%S')]
        RealDataDR =[3,0,1,2,5,1,0,1,2,1,0,0,0,0,1,1,1,15,34,75,190,256,161,130,254,449,427,909
                         ,595,686,600,516,438,518,483,367,248,131,242,114,110,107,76,74,84,93,152,87,
                         147,98,64,76,100]
        RealDataCum=[15,15,16,18,23,24,24,25,27,28,28,28,28,28,29,30,31,46,80,155,345,601,762,892,
                         1146,1595,2022,2931,3526,4212,4812,5328,5766,6284,6767,7134,7382,7513,7755,7869,
                         7979,8086,8162,8236,8320,8413,8565,8652,8799,8897,8961,9037,9137,9241,9332,9478,9583]
    if country == "United Kingdom":
        x1 = 113.26598
        x2= 0.163222 
        x3 = 1
        Mu = 0.06
        S0 = 66.44e6 #number of susceptible at time t0
        N = 52
        v = 1/30
        eta = 1/31
        # data from Feb. 13 until April 26
        s_date = [datetime.datetime.strptime('13/02/20 01:55:19', '%d/%m/%y %H:%M:%S')]
        RealDataDR =[0,0,0,0,0,2,0,0,0,1,0,1,0,0,4,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2,5,4,8,12,5,22,40,55,56,51,81,60,57,148,259,406,484,478,361,442,611,769,999,
                     1055,1255,1198,1378,2338,2375,2692,3087,3197,2822,2858,4273,4514,4913,4868,4911,4020,3592,5282,5450,5131,
                     4858,4313,3579,3489,4178,4326,5065,5292]
        RealDataCum=[2,2,2,2,3,3,4,4,4,8,9,9,10,10,10,10,10,10,10,10,10,10,11,11,11,13,18,22,30,42,47,69,109,164,220,271,352,412,469,617,876,
                     1282,1766,2244,2605,3047,3658,4427,5426,6481,7736,8934,10312,12650,15025,17717,20804,24001,26823,
                     29681,33954,38468,43381,48249,53160,57180,60772,66054,71504,76635,81493,85806,89385,92874,97052,101378,
                     106443,111735,116691,121412,125265,130119,134879,140366,145524,150494,154242,157715,162421,167150]
        
    v1 = f*v;
    v2 = (1-f)*v;
    t0 = (1/x2) - math.log(x1);          #initial time for the beginning of the early exponential growth phase
    return s_date, N, t0, v, eta, v1, v2, f, x1, x2, x3, Mu, RealDataDR, RealDataCum, S0



## Plotting the solution

In [20]:
def plotsiru(Country, f, days):
    """
    Plots the predicted SIR model results for a given list of countries and reproduction fraction for a number of days.

    Parameters:
    -----------
    country : List[str]
        A list of countries for which to plot the SIR model results.
    f : float
        The reproduction fraction used in the SIR model.
    days : int
        The number of days for which to predict SIR model results.

    Returns:
    --------
    None
        This function only generates a plot and does not return any values.
    """
    #initialize parameters based on the country
    s_date, N, t0, v, eta, v1, v2, f, x1, x2, x3, Mu, RealDataDR, RealDataCum, S0 = init_param(Country,f)
    
    t = np.linspace(0, days, days)     # Grid of time points (in days)
    # converting day numbers into date in order to show in the x axis
    for d in range(len(t)):
        s_date.append(s_date[0] + datetime.timedelta(days=d))
        
    #Cumulative graph
    CR, CU = Calc_Cum(t, S0, x2, v, eta, v1, v2, f, t0, N, Mu, x1)
    figCum = go.Figure()
    figCum.add_trace(go.Scatter(x=s_date, y=CR, name='CR(t)', mode = "lines", line = dict(color = "blue", shape = "spline", smoothing = 1))) 
    figCum.add_trace(go.Scatter(x=s_date, y=CU, name='CU(t)', mode = "lines", line = dict(color = "green", shape = "spline", smoothing = 1)))
    figCum.add_trace(go.Scatter(x=s_date, y=RealDataCum, name='Cumulative Data', mode = "markers", line = dict(color = "black", shape = "spline", smoothing = 1)))
    figCum.update_layout(autosize=False, width= 900, height=500, title='Cumulative Data ({})'.format(Country), yaxis_title='Individuals')
    
    #Daily reported cases graph
    DR, RealDataDR = Calc_Daily(t, S0, x2, v, eta, v1, v2, f, RealDataDR, N, Mu)
    figDaily = go.Figure()
    figDaily.add_trace(go.Scatter(x=s_date, y=DR, name='DR(t)', mode = "lines", line = dict(color = "purple", shape = "spline", smoothing = 1)))
    figDaily.add_trace(go.Scatter(x=s_date, y=RealDataDR, name='Daily Data', mode = "markers", line = dict(color = "black", shape = "spline", smoothing = 1)))
    #figDaily.add_trace(go.Bar(x=t, y=RealDataDR, name='Daily Data'))
    figDaily.update_layout(autosize=False, width= 900, height=500,title='Daily Data ({})'.format(Country), yaxis_title='Individuals')
    
    
    # epidemic curve graph
    S, I, R, U = Calc_All(t, S0, x2, v, eta, v1, v2, f, N, Mu)
    figAll = go.Figure()
    figAll.add_trace(go.Scatter(x=s_date, y=I, name='I(t)', mode = "lines", line = dict(color = "Red", shape = "spline", smoothing = 1)))
    figAll.add_trace(go.Scatter(x=s_date, y=R, name='R(t)', mode = "lines", line = dict(color = "blue", shape = "spline", smoothing = 1)))
    figAll.add_trace(go.Scatter(x=s_date, y=U, name='U(t)', mode = "lines", line = dict(color = "green", shape = "spline", smoothing = 1)))
    figAll.update_layout(autosize=False, width= 900, height=500, title='Epidemic Curve ({})'.format(Country), yaxis_title='Individuals')
    
    
    #transmission rate over time
    ti_overtime = Ti_over_time(t, N, x2, v1, v2, S0, eta, Mu)
    figTi = go.Figure()
    figTi.add_trace(go.Scatter(x=s_date, y=ti_overtime, name='Transmission rate (t)', mode = "lines", line = dict(color = "blue", shape = "spline", smoothing = 1))) 
    figTi.update_layout(autosize=False, width= 900, height=500, title='Transmission Rate ({})'.format(Country))
    
    # basic reproduction number over time
    R_0_overtime = R_0_over_time(t, N, x2, v1, v2, S0, eta, Mu)
    figR0 = go.Figure()
    figR0.add_trace(go.Scatter(x=s_date, y=R_0_overtime, name='Basic reproduction number (t)', mode = "lines", line = dict(color = "green", shape = "spline", smoothing = 1))) 
    figR0.update_layout(autosize=False, width= 900, height=500, title='Basic reproduction number over time ({})'.format(Country))

    
    ti0 = Ti_0(x2, v1, v2, S0, eta)
    print('\N{GREEK SMALL LETTER Tau}\N{SUBSCRIPT ZERO} = ' + "{:.2E}".format(ti0))
    print('R\N{SUBSCRIPT ZERO} = ' + "{:.2f}".format(R_0(ti0,v1, v2, S0, eta)))
    figR0.show()
    figTi.show()
    figCum.show()
    figDaily.show()
    figAll.show()

## Interactions

In [21]:
RepFraction = wg.FloatSlider(description='Fraction',value=0.1,min=0.1, max=0.9,step=0.1) 
predicDays = wg.IntSlider(description = 'Days ', value = 180, min = 50, max = 365, step = 1)
interact(plotsiru ,Country = ['Germany','Italy','South Korea','United Kingdom'], f= RepFraction, days = predicDays)

interactive(children=(Dropdown(description='Country', options=('Germany', 'Italy', 'South Korea', 'United King…

<function __main__.plotsiru(Country, f, days)>